In [316]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [317]:
# Habilita compatiblidad hacia atrás
import tensorflow._api.v2.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [318]:
# Base
base_image_path = Path.cwd() / Path("content-propio/Toto_small.jpg")
# base_image_path = Path.cwd() / Path("content-propio/guitar_small.jpg")
# base_image_path = Path.cwd() / Path("content-propio/facultad_small.jpg")
# base_image_path = Path.cwd() / Path("content-propio/fondo_small.jpg")
base_image_path = Path.cwd() / Path("content-propio/story_small.jpg")
# base_image_path = Path.cwd() / Path("content-propio/achiras_pelota_small.jpg")
# base_image_path = Path.cwd() / Path("content/775px-Neckarfront_Tübingen_Mai_2017.jpg")

# Style
# style_reference_image_path = Path.cwd() / Path("content/La_noche_estrellada1.jpg")
# style_reference_image_path = Path.cwd() / Path("content-propio/nieve_small.jpg")
style_reference_image_path = Path.cwd() / Path("content-propio/cubism.png")
# style_reference_image_path = Path.cwd() / Path("content-propio/kandinsky_small.jpg")

result_prefix = Path.cwd() / Path("content-propio/output")

In [319]:
# Definimos el tamaño de las imágenes a utilizar (fija el tamaño de salida)
# width, height = load_img(base_image_path).size
# img_nrows = 400
# img_ncols = int(width * img_nrows / height)
# print(img_nrows, img_ncols)

In [320]:
# hiperparámetros

total_variation_weight = 0.01
# total_variation_weight = 0.5
# total_variation_weight = 0.1

# style_weight = 1000
# style_weight = 5
# style_weight = 50
# style_weight = 1
style_weight = 10

content_weight = 1
# content_weight = 10
# content_weight = 100

# iterations = 100
# iterations = 300
# iterations = 3
iterations = 10

In [321]:
# Definimos el tamaño de las imágenes a utilizar (se conserva tamaño original)
width, height = load_img(base_image_path).size
img_nrows = height
img_ncols = width
print(img_nrows, img_ncols)

497 700


In [322]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [323]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [324]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [325]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

In [326]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [327]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

2023-08-28 17:41:14.549645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-28 17:41:14.597501: W tensorflow/c/c_api.cc:304] Operation '{name:'block2_conv1_13/kernel/Assign' id:16447 op device:{requested: '', assigned: ''} def:{{{node block2_conv1_13/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](block2_conv1_13/kernel, block2_conv1_13/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-08-28 17:41:14.792799: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Model loaded.


2023-08-28 17:41:15.148527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [328]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [329]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [330]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [331]:
# Se define el 1.25 como constante (hyperparámetro)
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [332]:
REF_BASE_IMG = 0
REF_STYLE_IMG = 1
REF_COMB_IMG = 2

# Armamos la loss total
loss = K.variable(0.0)

layer_features = outputs_dict['block5_conv2']

base_image_features = layer_features[REF_BASE_IMG, :, :, :]
combination_features = layer_features[REF_COMB_IMG, :, :, :]

loss = loss + content_weight * content_loss(base_image_features,combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[REF_STYLE_IMG, :, :, :] 
    combination_features = layer_features[REF_COMB_IMG, :, :, :]
    
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl

loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [333]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

In [334]:
def eval_loss_and_grads(x):
    # Reshape the input image to match the expected input shape for the model
    x = x.reshape((1, img_nrows, img_ncols, 3))
    # Compute the loss and gradients for the input image
    outs = f_outputs([x])
    # Extract the loss value from the computed outputs
    loss_value = outs[0]
    # If there is only one gradient value, flatten it and convert to float64
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    # If there are multiple gradient values, convert them to a flat numpy array of float64
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    # Return the computed loss and gradient values
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [335]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [336]:
# ! mkdir -p content/output
! mkdir -p content-propio/output

In [337]:
print("Content:", base_image_path.name.split(".")[0])
print("Style:", style_reference_image_path.name.split(".")[0])
print("total_variation_weight:", total_variation_weight)
print("style_weight:", style_weight)
print("content_weight:", content_weight)
print("Iterations:", iterations)

Content: story_small
Style: cubism
total_variation_weight: 0.01
style_weight: 1
content_weight: 100
Iterations: 10


In [338]:
BASEIMG = base_image_path.name.upper().split(".")[0]
STYLEIMG = style_reference_image_path.name.upper().split(".")[0]
TOTALVARIATIONW = str(total_variation_weight)
STYLEW = str(style_weight)
CONTENTW = str(content_weight)
IT_NUMBER = str(1)
# print(f'output_{BASEIMG}_{STYLEIMG}_{TOTALVARIATIONW}_{STYLEW}_{CONTENTW}_at_iteration_{IT_NUMBER}.png')

In [339]:
evaluator = Evaluator() # 

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    # fname = result_prefix / ('output_at_iteration_%d.png' % i)
    fname = result_prefix / (f'output_{BASEIMG}_{STYLEIMG}_{TOTALVARIATIONW}_{STYLEW}_{CONTENTW}_at_iteration_{i}.png')
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0


2023-08-28 17:41:16.749769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-28 17:41:16.782121: W tensorflow/c/c_api.cc:304] Operation '{name:'Variable_41/Assign' id:16909 op device:{requested: '', assigned: ''} def:{{{node Variable_41/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](Variable_41, Variable_41/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-08-28 17:41:16.980612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-28 17:41:17.060349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enable

Current loss value: 7060884500.0
Image saved as /Users/hordia/dev/ML/course-ITBA-DL/course-notebooks-labs/TP-Final-Computer-Vision/hordia-Trabajo-Final-CV/content-propio/output/output_STORY_SMALL_CUBISM_0.01_1_100_at_iteration_0.png
Iteration 0 completed in 13s
Start of iteration 1
Current loss value: 6069878300.0
Image saved as /Users/hordia/dev/ML/course-ITBA-DL/course-notebooks-labs/TP-Final-Computer-Vision/hordia-Trabajo-Final-CV/content-propio/output/output_STORY_SMALL_CUBISM_0.01_1_100_at_iteration_1.png
Iteration 1 completed in 11s
Start of iteration 2
Current loss value: 5209514500.0
Image saved as /Users/hordia/dev/ML/course-ITBA-DL/course-notebooks-labs/TP-Final-Computer-Vision/hordia-Trabajo-Final-CV/content-propio/output/output_STORY_SMALL_CUBISM_0.01_1_100_at_iteration_2.png
Iteration 2 completed in 11s
Start of iteration 3
Current loss value: 4530601000.0
Image saved as /Users/hordia/dev/ML/course-ITBA-DL/course-notebooks-labs/TP-Final-Computer-Vision/hordia-Trabajo-Final

<center>

| Content | Style |
| --- | --- |
| <img src='./content-propio/achiras_pelota_small.jpg'> | <img src='./content/La_noche_estrellada1.jpg'> |


| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.01 | 5 | 1 | 100 |


<img src='./resultados/PUNTO-9/output_PELOTA_NOCHE-ESTRELLADA_0.01_5_1_iteration_99.png'>

</center>
